In [73]:
import numpy as np
import scipy as sp
import scipy.integrate as integrate


def toK(c):
    return (c + 273.15)

def toPa(bar):
    return (bar * 1e5)
    
R = 8.314462618 # J/(K·mol)
R_Bar =  0.00008314462618 # bar·m³/(K·mol)
Tpad = 298.15

class Comp:
    def __init__(self, omega, Pc, Tb, Tc, NCParams, Ant1 = None, Ant2 = None, Ant3 = None): # NCParams: A, B, C, D cp do V.Ness; Ant: A, B, C 
        self.omega = omega
        self.Pc = Pc
        self.Tb = Tb
        self.Tc = Tc
        self.NCParams = NCParams
        self.Ant1 = Ant1
        self.Ant2 = Ant2
        self.Ant3 = Ant3
    
    def Ant(self, T): # faixas de antoine
        if T > self.Ant1[3] and T < self.Ant1[4]:
            return self.Ant1[:3]

        if T > self.Ant2[3] and T < self.Ant2[4]:
            return self.Ant2[:3]

        if T > self.Ant3[3] and T < self.Ant3[4]:
            return self.Ant3[:3]

    def Antoine(self, T): # NIST, T em K
        Ant = self.Ant(T)
        AntA = Ant[0]
        AntB = Ant[1]
        AntC = Ant[2]
        return 10 ** (AntA - (AntB / (T + AntC)))

    def TAntoine(P):
        

    def Estado(self, T, P):
        if (T > self.Tc and P > self.Pc) or self.Ant1 == None:
            return "gas"
        elif T < self.Tc and P > self.Pc:
            return "liquid"
        elif T < self.Tc and P < self.Pc:
            if  P < self.Antoine(T):
                return "vapour"
            else:
                return "liquid"

    def Cp_ig(self, T):
        A = self.NCParams[0]
        B =  self.NCParams[1]
        C = self.NCParams[2]
        D =  self.NCParams[3]
        return R * (A + (B * T) + (C * (T ** 2)) + (D * (T ** (-2))))
    
    def Cp_L(self, T):
        return R * (1.45 + (0.45/(1-(T / self.Tc)) + 0.25 * self.omega * (17.11 + (25.2 * (1 - (T / self.Tc))**(1/3)) / (T / self.Tc) + (1.742)/(1 - (T / self.Tc)) ) ) ) + Cp_ig(T)
    
    def Hres(self, P, T):
        self.Tc = toPa(self.Tc)
        P = toPa(P)
        k = 0.37464 + (0.154226 * self.omega) - (0.26992 * (self.omega ** 2))
        alphat = (1 + k * (1 - np.sqrt(T/self.Tc))) ** 2
        ac = 0.45724 * (((R ** 2) * (self.Tc ** 2)) / self.Tc )
        a = ac * alphat
        b = 0.07780 * R * (self.Tc / self.Tc)
        A = (a * P) / ((R ** 2) * (T ** 2.5))
        B =  (b * P) / (R * T)
        alpha = -1 + B
        beta = A - (3 * B ** 2 ) - 2 * B
        gamma = - (A * B) + (B ** 2) + (B ** 3)
        Z = np.roots([1, alpha, beta, gamma])
        Z = Z[np.isreal(Z)]
        Z = max(Z)
        Tdadt = - ac * k * alphat * (T / self.Tc)
        h = (b * P) / (Z * R * T)
        return R * T * (  (Z - 1) - ((a - Tdadt)/(2 * np.sqrt(2) * b * R * T)) * np.log((1 + h * (1 + np.sqrt(2)))/(1 + h * (1 - np.sqrt(2))))  )

    def Hvap(self):
        return 1.093 * R * self.Tc * ((self.Tb/self.Tc) * ((np.log(toPa(self.Pc) - 1.013))/ (0.930 - (self.Tb/self.Tc))))

    def H(self, P, T):
        self.Estado(P, Tpad) # estado inicial


H2 = Comp( -0.217, # Prausnitz
          12.93, #Bar
          20.27, #K
          32.98, #K
         [3.249,
         0.422e3,
         0,
         0.083e-5])

CO = Comp(
        0.045, # Prausnitz
        34.94, #Bar
        81.66, #K
        132.85, #K
        [ 3.376,
         0.557e3,
         0,
         -0.031e-5])

CO2 = Comp(
        0.225, # Prausnitz
        73.74, #Bar
        194.69, #K
        304.12, #K
        [ 5.457,
        1.045e3,
        0,
        -1.157e-5]) 

N2 = Comp(
        0.037,
        33.98,
        77.35,
        126.20,
        [3.280,
        0.593e3,
        0,
        0.040e-5])
    
CH3OH = Comp(
        0.565,
        80.97,
        337.69,
        512.64,
        [2.211, # A
        12.216e3, # B
        -3.450e6, # C
        0], # D
        [5.15853, 1569.613, -34.846, 353.5, 512.63], # A, B, C, D, Tmin, Tmax Ambrose 1975
        [5.20409, 1581.341, -33.50, 288.1, 356.83], # Ambrose 1970
        [5.31301, 1676.569, -21.728, 353, 483]) # Hirata 1967

H2O = Comp(
        0.344, # Prausnitz
        220.64, #Bar
        373.15, #K
        647.14, #K
        [3.470, # V.Ness - |
        1.450e3,
        0,
        0.121e-5],
        [3.55959, 643.748, -198.043, 373, 573], # A, B, C, D, Tmin, Tmax do NIST liu e lindsey
        [4.6543, 1435.264, -64.848, 255.9, 373] # Stull, 1947
        )


def Hm(T, P, Comp):

    #if Comp.Pc < P and T < Comp.Tc: # é gás portanto


    pass

liquid
[5.20409, 1581.341, -33.5]
1.0313566911995582


In [75]:
# Temperatura Corrente 3

# Entalpia Corrente 2
T2 = toK(163)
P2 = 52
#N2
H_2N2 = integrate.quad(Cp_ig, Tpad, T2, args=
(Comp["N2"]["A"],
Comp["N2"]["B"],
Comp["N2"]["C"],
Comp["N2"]["D"],
))[0] + Hres(P2, T2, Comp["N2"]["Tc"],  Comp["N2"]["Pc"], Comp["N2"]["omega"])
#CO
H_2CO = integrate.quad(Cp_ig, Tpad, T2, args=
(Comp["CO"]["A"],
Comp["CO"]["B"],
Comp["CO"]["C"],
Comp["CO"]["D"],
))[0] + Hres(P2, T2, Comp["CO"]["Tc"],  Comp["CO"]["Pc"], Comp["CO"]["omega"])
#CO2
H_2CO2 = integrate.quad(Cp_ig, Tpad, T2, args=
(Comp["CO2"]["A"],
Comp["CO2"]["B"],
Comp["CO2"]["C"],
Comp["CO2"]["D"],
))[0] + Hres(P2, T2, Comp["CO2"]["Tc"],  Comp["CO2"]["Pc"], Comp["CO2"]["omega"])
#H2
H_2H2 = integrate.quad(Cp_ig, Tpad, T2, args=
(Comp["H2"]["A"],
Comp["H2"]["B"],
Comp["H2"]["C"],
Comp["H2"]["D"],
))[0] + Hres(P2, T2, Comp["H2"]["Tc"],  Comp["H2"]["Pc"], Comp["H2"]["omega"])
# Hm_2 = H2CO + H2CO2 + H2H2 + H2N2 
# print(f'{Hm_2 / 1000} kJ / mol')

# Entalpia Corrente 12
T12 = toK(22)
P12 = 52
H_12N2 = integrate.quad(Cp_ig, Tpad, T12, args=
(Comp["N2"]["A"],
Comp["N2"]["B"],
Comp["N2"]["C"],
Comp["N2"]["D"],
))[0] + Hres(P12, T12, Comp["N2"]["Tc"],  Comp["N2"]["Pc"], Comp["N2"]["omega"])
#CO
H_12CO = integrate.quad(Cp_ig, Tpad, T12, args=
(Comp["CO"]["A"],
Comp["CO"]["B"],
Comp["CO"]["C"],
Comp["CO"]["D"],
))[0] + Hres(P12, T12, Comp["CO"]["Tc"],  Comp["CO"]["Pc"], Comp["CO"]["omega"])
#CO2
H_12CO2 = integrate.quad(Cp_ig, Tpad, T12, args=
(Comp["CO2"]["A"],
Comp["CO2"]["B"],
Comp["CO2"]["C"],
Comp["CO2"]["D"],
))[0] + Hres(P12, T12, Comp["CO2"]["Tc"],  Comp["CO2"]["Pc"], Comp["CO2"]["omega"])
#H2
H_12H2 = integrate.quad(Cp_ig, Tpad, T12, args=
(Comp["H2"]["A"],
Comp["H2"]["B"],
Comp["H2"]["C"],
Comp["H2"]["D"],
))[0] + Hres(P12, T12, Comp["H2"]["Tc"],  Comp["H2"]["Pc"], Comp["H2"]["omega"])
#H2O
print(f' {P12 < Comp["H2O"]["Pc"]} , {T12 < Comp["H2O"]["Tc"]}')
H_12H2O = integrate.quad(Cp_ig, Tpad, T12, args=
(Comp["H2O"]["A"],
Comp["H2O"]["B"],
Comp["H2O"]["C"],
Comp["H2O"]["D"],
))[0]
H_12H2O = 
# Hm_12 = H_12CO + H_12CO2 + H_12H2 + H_12N2 

SyntaxError: invalid syntax (1627298210.py, line 75)

In [ ]:
Hig= integrate.quad(Cp_ig, Tpad, Comp["H2O"]["Tb"], args=
(Comp["H2O"]["A"],
Comp["H2O"]["B"],
Comp["H2O"]["C"],
Comp["H2O"]["D"],
))

HL_h2o = integrate.quad(Cp_L, Tpad, Comp["H2O"]["Tb"], args=
(
    Comp["H2O"]["Tc"],
    Comp["H2O"]["omega"]
)
)

print(Hig)
print(HL_h2o)

Hr = Hres(40, toK(120), Comp["N2"]["Tc"],  Comp["N2"]["Pc"], Comp["N2"]["omega"])
H = Hig[0] + HL_h2o[0]

print(f'{H/1000} kJ/mol')

(303496158.6672187, 3.3694842323763597e-06)
(4655.170180889767, 5.168277118372274e-11)
303500.8138373996 kJ/mol


In [ ]:
# teste cp medio
Hig= integrate.quad(Cp_ig, Tpad, toK(120), args=
(Comp["N2"]["A"],
Comp["N2"]["B"],
Comp["N2"]["C"],
Comp["N2"]["D"],
))[0]
Higmed = ((Cp_ig(toK(120), Comp["N2"]["A"],
Comp["N2"]["B"],
Comp["N2"]["C"],
Comp["N2"]["D"]) + Cp_ig(Tpad, Comp["N2"]["A"],
Comp["N2"]["B"],
Comp["N2"]["C"],
Comp["N2"]["D"]))/2) * (toK(120)-Tpad)

print(Hig - Higmed)

0.0


In [ ]:
import chemicals.heat_capacity as hc


water_low_gas_coeffs = [30.09200, 6.832514/1e3, 6.793435/1e6, -2.534480/1e9, 0.082139*1e6]
gi = hc.Shomate_integral(toK(100), *water_low_gas_coeffs) - hc.Shomate_integral(toK(25), *water_low_gas_coeffs)

print(integrate.quad(hc.Rowlinson_Bondi, 0, toK(100), (Comp["H2O"]["Tc"], Comp["H2O"]["omega"], gi))[0] / 1000)

1441.1022814253024


C:\Users\luizd\AppData\Local\Temp\ipykernel_17324\798857567.py:7: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  print(integrate.quad(hc.Rowlinson_Bondi, 0, toK(100), (Comp["H2O"]["Tc"], Comp["H2O"]["omega"], gi))[0] / 1000)
